In [ ]:
import os
import glob
import enum
import time
import random
import multiprocessing
from pathlib import Path

import torch
import torchvision
import torchio as tio
import torch.nn.functional as F

import numpy as np
from unet import UNet
from scipy import stats
from sklearn.model_selection import KFold, train_test_split
from torch.utils.data import DataLoader, SubsetRandomSampler
from matplotlib import cm
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import seaborn as sns

from IPython import display
from tqdm.auto import tqdm

device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
num_workers = 0
batch_size = 1
k = 5                # num k-folds
seed = 88
random.seed(seed)
torch.manual_seed(seed)

In [ ]:
print(torch.cuda.is_available())

Load files

In [ ]:
file_dir = "./dataset"
raw_dir = r'{0}/raw/*.nii'.format(file_dir)
label_dir = r'{0}/label/*.nii'.format(file_dir)

image_paths = sorted(glob.glob(raw_dir))
label_paths = sorted(glob.glob(label_dir))
assert len(image_paths) == len(label_paths)

subjects = []
subjects_full = []
i = 0
for (image_path, label_path) in zip(image_paths, label_paths):
    subject = tio.Subject(
        img = tio.ScalarImage(image_path),
        label = tio.LabelMap(label_path),
    )
    subjects.append(subject)
#     subjects_full.append(subject)
#     if i in [5,10,15,20,25,30,40,50,60,70]:
#         subjects.append(subject)
#     i+=1
# dataset_full = tio.SubjectsDataset(subjects_full)
# print('Dataset full size:', len(dataset_full), 'subjects')

dataset = tio.SubjectsDataset(subjects)
print('Dataset size:', len(dataset), 'subjects')

Visualisation functions

In [ ]:
def rotate(image, n = 1):
    # Rotate for visualization purposes
    image = np.rot90(image, n)
    image = np.fliplr(image)
    return image

def plot_slice(subject, img_name, mask_names = None, indices = None):
    """
    Plots CT slices with optional mask overlays
    
    :param subject tio.Subject: subject containing images & masks
    :param img_name str: name of base image in subject
    :param mask_names [str]: list of names of masks to overlay on top of base image
    :param indices [int]: list of indices to override default slice indices set at midpoint of axes 
    """
    show_mask = False
    img = subject[img_name].data[0,:,:,:]
    sr, sa, ss = subject[img_name].spacing                # for calculating aspect ratio
    
    ncol = 1
    if mask_names != None:
        show_mask = True
        masks = [subject[mask_name].data[0,:,:,:] for mask_name in mask_names]
        ncol += len(masks)
        for x in masks:
            assert img.shape == x.shape
    
    if indices == None:
        indices = np.array(img.shape) // 2          # find midpoint of image along each axis
    
    nrow = 3
    plt.figure(figsize=(9, 8))
    sns.set_style("ticks")
    cmaps = ['Set1', 'Set2']
    
    for i, indice in enumerate(indices):            # plot slice along each axis
        if i == 0:
            aspect = ss / sa
            img_slice = img[indice, :, :]
            plt.subplot(nrow, ncol, ncol*i+1)
            plt.title(img_name)
            plt.ylabel("sagittal")
            plt.imshow(rotate(img_slice), aspect=aspect, cmap='gray')
            if show_mask:
                for j in range(2, ncol+1):
                    plt.subplot(nrow, ncol, ncol*i+j)
                    plt.title(mask_names[j-2])
                    mask_slice = np.ma.masked_where(masks[j-2][indice, :, :] == 0, masks[j-2][indice, :, :])
                    plt.imshow(rotate(img_slice), aspect=aspect, cmap='gray')
                    plt.imshow(rotate(mask_slice), aspect=aspect, cmap=cmaps[j-2], alpha=0.7)
        elif i == 1:
            aspect = ss / sr
            img_slice = img[:, indice, :]
            plt.subplot(nrow, ncol, ncol*i+1)
            plt.ylabel("coronal")
            plt.imshow(rotate(img_slice), aspect=aspect, cmap='gray')
            if show_mask:
                for j in range(2, ncol+1):
                    plt.subplot(nrow, ncol, ncol*i+j)
                    mask_slice = np.ma.masked_where(masks[j-2][:, indice, :] == 0, masks[j-2][:, indice, :])
                    plt.imshow(rotate(img_slice), aspect=aspect, cmap='gray')
                    plt.imshow(rotate(mask_slice), aspect=aspect, cmap=cmaps[j-2], alpha=0.7)
        else:
            aspect = sa / sr
            img_slice = img[:, :, indice]
            plt.subplot(nrow, ncol, ncol*i+1)
            plt.ylabel("axial")
            plt.imshow(rotate(img_slice), aspect=aspect, cmap='gray')
            if show_mask:
                for j in range(2, ncol+1):
                    plt.subplot(nrow, ncol, ncol*i+j)
                    mask_slice = np.ma.masked_where(masks[j-2][:, :, indice] == 0, masks[j-2][:, :, indice])
                    plt.imshow(rotate(img_slice), aspect=aspect, cmap='gray')
                    plt.imshow(rotate(mask_slice), aspect=aspect, cmap=cmaps[j-2], alpha=0.7)
    
    plt.tight_layout()
    sns.despine()
    plt.show()

In [ ]:
# for i, subject in enumerate(dataset):
#     if i>1:
#         break
#     toCanon = tio.ToCanonical()
#     subject = toCanon(subject)
#     print("Subject {0}".format(i+1))
#     plot_slice(subject, "img", ["label"], indices = [280, 430, 50])

Preprocessing
* Orient all images to canonical (RAS+) orientation
* Resize images to size (256,256,128)
* Perform histogram standardization of intensity values
* Perform Z-normalization
* Encode labels as '0' or '1'

Data augmentation (applied at random during training only)
* Flip along lateral axis
* Zoom out by -10 to +10%
* Rotate by -10° to +10°
* Translate along each axis by -10 to +10mm
* Downsample images by a factor of 1.5 to 5
* Add Gaussian noise

In [ ]:
# train histogram
landmarks_file = 'landmarks.npy'

if os.path.exists(landmarks_file):
    landmarks = np.load(landmarks_file)
else:
    landmarks = tio.HistogramStandardization.train(image_paths)
    np.save(landmarks_file, landmarks)

In [ ]:
# standardise labels to '0' or '1'
def binarize(x):
    return x > 0

train_transforms = tio.Compose([
    tio.ToCanonical(),
#     tio.Resample(2),
    tio.CropOrPad((256,256,128), mask_name='label'),
    tio.HistogramStandardization({'img': landmarks}),
    tio.ZNormalization(masking_method=tio.ZNormalization.mean),
    tio.OneOf({
        tio.RandomAnisotropy(): 0.5,
        tio.OneOf({
            tio.RandomAffine(
                scales = (0.1),
                degrees = (10),
                translation = (10)
            ): 0.2,
            tio.RandomFlip(
                axes=('LR',)
            ): 0.4,
            tio.RandomNoise(): 0.4,
        }): 0.3,
    }),
    tio.Lambda(binarize, types_to_apply=[tio.LABEL]),
    tio.OneHot(),
])

test_transforms = tio.Compose([
    tio.ToCanonical(),
    tio.CropOrPad((256,256,128), mask_name='label'),
    tio.HistogramStandardization({'img': landmarks}),
    tio.ZNormalization(masking_method=tio.ZNormalization.mean),
    tio.Lambda(binarize, types_to_apply=[tio.LABEL]),
    tio.OneHot(),
])

Split train/test set


In [ ]:
val_ratio = 0.15
test_ratio = 0.2
num_subjects = len(dataset)
num_val_subjects = int(val_ratio * num_subjects)
num_test_subjects = int(test_ratio * num_subjects)
num_training_subjects = num_subjects - num_val_subjects - num_test_subjects

training_subjects, validation_subjects, test_subjects = torch.utils.data.random_split(
    subjects, [num_training_subjects, num_val_subjects, num_test_subjects], generator=torch.Generator().manual_seed(seed))

training_set = tio.SubjectsDataset(
    training_subjects, transform=train_transforms)
validation_set = tio.SubjectsDataset(
    validation_subjects, transform=test_transforms)
test_set = tio.SubjectsDataset(
    test_subjects, transform=test_transforms)

print('Training set:', len(training_set), 'images')
print('Validation set:', len(validation_set), 'images')
print('Test set:', len(test_set), 'images')

# splits = KFold(n_splits = k, shuffle = True, random_state = seed)

<!-- Split k-folds -->

Build model

In [ ]:
CHANNELS_DIMENSION = 1
SPATIAL_DIMENSIONS = 2, 3, 4
num_epochs = 5
threshold = 0.5      # threshold for binary mask

class Action(enum.Enum):
    TRAIN = 'Training'
    VALIDATE = 'Validation'
    TEST = 'Testing'

class EarlyStopping():
    def __init__(self, patience = 10):
        self.patience = patience
        self.patience_count = 0
        self.best_loss = float('inf')
        self.best_epoch = -1
        self.best_model = None
        self.is_stopped = False

    def monitor(self, curr_loss, epoch, model):
        if curr_loss < self.best_loss:
            print("Model loss improved from {0:.5f} to {1:.5f}".format(self.best_loss, curr_loss))
            self.save_best(curr_loss, epoch, model)
        else:
            self.patience_count += 1
            if self.patience_count >= self.patience:
                self.is_stopped = True
                print("Model did not improve for {0} epochs. Best performance was achieved at epoch {1} with loss: {2:.5f}\nStopping training early".format(self.patience, self.best_epoch, self.best_loss))
            else:
                print("Model loss did not improve for {0}/{1} epochs".format(self.patience_count, self.patience))

        return self.is_stopped

    def save_best(self, curr_loss, epoch, model):
        self.best_loss = curr_loss
        self.best_epoch = epoch
        self.best_model = model
        self.patience_count = 0

    def get_best(self):
        return self.best_epoch, self.best_model
    
def prepare_batch(batch, device):
    inputs = batch['img'][tio.DATA].to(device)
    targets = batch['label'][tio.DATA].to(device)
    return inputs, targets

def get_dice_score(output, target, epsilon=1e-9):
    p0 = output
    g0 = target
    p1 = 1 - p0
    g1 = 1 - g0
    tp = (p0 * g0).sum(dim=SPATIAL_DIMENSIONS)
    fp = (p0 * g1).sum(dim=SPATIAL_DIMENSIONS)
    fn = (p1 * g0).sum(dim=SPATIAL_DIMENSIONS)
    num = 2 * tp
    denom = 2 * tp + fp + fn + epsilon
    dice_score = num / denom
    return dice_score

def get_dice_loss(output, target):
    return 1 - get_dice_score(output, target)

def get_model_and_optimizer(device):
    model = UNet(
        in_channels=1,
        out_classes=2,
        dimensions=3,
        num_encoding_blocks=3,
        out_channels_first_layer=8,
        normalization='batch',
        upsampling_type='linear',
        padding=True,
        activation='PReLU',
#         activation='ReLU',
    ).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
    return model, optimizer

def run_epoch(epoch_idx, action, loader, model, optimizer):
    is_training = action == Action.TRAIN
    epoch_losses = []
    epoch_accs = []
    times = []
    model.train(is_training)
    for batch_idx, batch in enumerate(tqdm(loader)):
        inputs, targets = prepare_batch(batch, device)
        optimizer.zero_grad()
        
        with torch.set_grad_enabled(is_training):
            logits = model(inputs)
            probabilities = torch.sigmoid(logits)
#             probabilities = torch.floor(torch.sigmoid(logits) + threshold)
#             probabilities = F.softmax(logits, dim=CHANNELS_DIMENSION)
            batch_losses = get_dice_loss(probabilities, targets)
            batch_loss = batch_losses.mean()
            if is_training:
                batch_loss.backward()
                optimizer.step()
            times.append(time.time())
            epoch_losses.append(batch_loss.item())
            
        batch_accuracy = get_dice_score((probabilities > threshold).float(), targets).mean()
        epoch_accs.append(batch_accuracy.item())
            
    epoch_losses = np.array(epoch_losses)
    epoch_loss = epoch_losses.mean()
    
    epoch_accs = np.array(epoch_accs)
    epoch_acc = epoch_accs.mean()
    
    if action == Action.TEST:
        epoch_loss_std = epoch_losses.std()
        epoch_acc_std = epoch_accs.std()
        print("{0} mean loss: {1:.5f} \u00B1 {2:.5f}".format(action.value, epoch_loss, epoch_loss_std))
        print("{0} mean accuracy: {1:.5f} \u00B1 {2:.5f}".format(action.value, epoch_acc, epoch_acc_std))
    else:
        print(f'{action.value} mean loss: {epoch_loss:0.5f}')
        print(f'{action.value} mean accuracy: {epoch_acc:0.5f}')
    return epoch_loss, epoch_acc

def train(num_epochs, training_loader, validation_loader, model, optimizer, early_stopper, weights_stem):
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
#     val_losses.append(run_epoch(0, Action.VALIDATE, validation_loader, model, optimizer))
    for epoch_idx in range(1, num_epochs + 1):
        print("\n=== Starting epoch {0} ===".format(epoch_idx))
#         train_losses.append(run_epoch(epoch_idx, Action.TRAIN, training_loader, model, optimizer))
#         val_losses.append(run_epoch(epoch_idx, Action.VALIDATE, validation_loader, model, optimizer))
        train_loss, train_acc = run_epoch(epoch_idx, Action.TRAIN, training_loader, model, optimizer)
        val_loss, val_acc = run_epoch(epoch_idx, Action.VALIDATE, validation_loader, model, optimizer)
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        val_losses.append(val_loss)
        val_accs.append(val_acc)
        
        # Saves current state of model
        torch.save(model.state_dict(), f'{weights_stem}_epoch_{epoch_idx}.pth')
        
        # Check early stopping
        if early_stopper != None:
            is_stopped = early_stopper.monitor(val_loss, epoch_idx, model.state_dict())
            if is_stopped:
                # Save best model
                best_epoch, best_model = early_stopper.get_best()
                torch.save(best_model, f'{weights_stem}_best.pth')
                break
    
    print("Training ended at epoch {0}".format(epoch_idx))    
    return np.array(train_losses), np.array(train_accs), np.array(val_losses), np.array(val_accs)

def test(test_loader, model, optimizer):
    loss, accuracy = run_epoch(1, Action.TEST, test_loader, model, optimizer)
    return loss, accuracy

# training_loader = torch.utils.data.DataLoader(
#     training_set,
#     batch_size=training_batch_size,
#     shuffle=True,
#     num_workers=num_workers,
# )

# validation_loader = torch.utils.data.DataLoader(
#     validation_set,
#     batch_size=validation_batch_size,
#     num_workers=num_workers,
# )

In [ ]:
# fold_train_loss = []
# fold_val_loss = []

# for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(training_set)))):
#     print("\n===== FOLD {0} ======".format(fold + 1))
       
#     model, optimizer = get_model_and_optimizer(device)
#     weights_path = 'whole_image_state_dict.pth'
#     weights_stem = 'whole_images'
    
#     # Load in training & validation set for the current fold
#     train_sampler = SubsetRandomSampler(train_idx)
#     validation_sampler = SubsetRandomSampler(val_idx)   
#     train_loader = DataLoader(training_set, batch_size=batch_size, sampler=train_sampler)
#     validation_loader = DataLoader(training_set, batch_size=batch_size, sampler=validation_sampler)
    
#     # Train
#     train_losses, val_losses = train(num_epochs, train_loader, validation_loader, model, optimizer, weights_stem)
#     fold_train_loss.append(np.mean(train_losses))
#     fold_val_loss.append(np.mean(val_losses))
    
#     print("\n=== Fold {0}\nTrain loss: {1}\nVal loss: {2}".format(fold+1, fold_train_loss[-1], fold_val_loss[-1]))

# print("\n===== END TRAINING =====")

Train model

In [ ]:
model, optimizer = get_model_and_optimizer(device)
weights_path = 'whole_image_state_dict.pth'
weights_stem = 'whole_images'
train_loader = torch.utils.data.DataLoader(
    training_set,
    batch_size = batch_size,
    shuffle = True,
    num_workers = num_workers
)
validation_loader = torch.utils.data.DataLoader(
    validation_set,
    batch_size = batch_size,
    num_workers = num_workers
)
early_stopper = EarlyStopping(patience = 10)

train_losses, train_accs, val_losses, val_accs = train(
    num_epochs, train_loader, validation_loader, model, optimizer, early_stopper, weights_stem
)
checkpoint = {
    'train_losses': train_losses,
    'val_losses': val_losses,
    'weights': model.state_dict(),
}
torch.save(checkpoint, weights_path)

Plot training & validation losses

In [ ]:
sns.set(font_scale=1)
sns.set_style("ticks")
sns.set_palette(sns.color_palette("Set2"))

plt.figure(0)
plt.plot(range(1, len(train_losses) + 1), train_losses, label="Training Loss")
plt.plot(range(1, len(train_losses) + 1), val_losses, label="Validation Loss")
plt.xticks(range(1, len(train_losses) + 1))
plt.legend()
plt.title("Losses")
plt.xlabel("Epoch")
plt.ylabel("Loss")
sns.despine()
# plt.show()


# plt.clf()
plt.figure(1)
plt.plot(range(1, len(train_losses) + 1), train_accs, label="Training DSC")
plt.plot(range(1, len(train_losses) + 1), val_accs, label="Validation DSC")
plt.xticks(range(1, len(train_accs) + 1))
plt.ylim((0, 1))
plt.legend()
plt.title("Dice Score Coefficient (DSC)")
plt.xlabel("Epoch")
plt.ylabel("DSC")
sns.despine()

# plt.show()

Load trained model

In [ ]:
trained_model = UNet(
    in_channels=1,
    out_classes=2,
    dimensions=3,
    num_encoding_blocks=3,
    out_channels_first_layer=8,
    normalization='batch',
    upsampling_type='linear',
    padding=True,
    activation='PReLU',
).to(device)
trained_model.load_state_dict(torch.load("./whole_images_best.pth"))
trained_model.eval()
optimizer = torch.optim.AdamW(trained_model.parameters(), lr=1e-3)

Test trained model on test set

In [ ]:
test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=batch_size,
    num_workers=num_workers,
)

# test_loss = run_epoch(0, Action.VALIDATE, test_loader, model, optimizer)
test_loss, test_acc = test(test_loader, trained_model, optimizer)

Visualise predictions on test set

In [ ]:
test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=1,
    num_workers=num_workers,
)
generator = iter(test_loader)

for i in range(len(test_loader)):
    print("===================\nSegmenting image {0}".format(i+1))
    batch = next(generator)     
    inputs, targets = prepare_batch(batch, device)
    
    with torch.no_grad():
        probabilities = torch.sigmoid(trained_model(inputs)).cpu()
        
    inputs = inputs.cpu()
    targets = targets.cpu()
    truth = torch.argmax(targets[0], dim=0, keepdim=True)
    predicted = torch.argmax((probabilities[0]>threshold).float(), dim=0, keepdim=True)
    affine = batch['label'][tio.AFFINE][0].numpy()
    
    subject = tio.Subject(
        ct=tio.ScalarImage(tensor=inputs[0], affine=affine),
        truth=tio.LabelMap(tensor=truth, affine=affine),
        predicted=tio.LabelMap(tensor=predicted, affine=affine),
    )
    
    score = get_dice_score((probabilities > threshold).float(), targets).mean().item()
    print("Dice score: {0:.5f}".format(score))
    
    plot_slice(subject, "ct", ["truth", "predicted"])

In [ ]:
for i, subject in enumerate(test_set.dry_iter()):
    subject['img'].save("./testset88/testimg{0}.nii".format(i+1))
    subject['label'].save("./testset88/testlabel{0}.nii".format(i+1))